# Prérequis

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import csv



# La liste des acteurs par film

## Création des Dataframes

In [ ]:
df_noms = pd.read_table("./data/name.basics.tsv", sep='\t')
df_film = pd.read_table("./data/title.principals.tsv", sep='\t')
df_titre = pd.read_table("./data/title.basics.tsv", sep='\t')

## Optimisation / Modification des Dataframes

In [ ]:
### Optimisation Dataframe "noms"

df_noms.info(memory_usage="deep")
df_noms = df_noms.drop(["knownForTitles","birthYear","deathYear"], axis = 1)
df_noms.head()
df_noms = df_noms.dropna()
df_noms = df_noms[["nconst","primaryName"]][df_noms["primaryProfession"].str.contains("actor|actress", regex=True)]
df_noms_clean = df_noms
del df_noms
df_noms_clean.info(memory_usage="deep")

### Optimisation Dataframe "film"

df_film.info(memory_usage="deep")
df_film = df_film.drop(["ordering","job","characters"], axis = 1)
df_films = df_film.dropna()
del df_film
df_films = df_films[["tconst","nconst"]][df_films["category"].str.contains("actor|actress", regex=True)]
df_films_clean = df_films
del df_films
df_films_clean.info(memory_usage="deep")

### Optimisation Dataframe "titre"

df_titre.info(memory_usage="deep")
df_titre = df_titre.drop(["titleType","originalTitle","isAdult","startYear","endYear","runtimeMinutes","genres"], axis = 1)
df_titre_clean = df_titre.dropna()
del df_titre
df_titre_clean.info(memory_usage="deep")

## Fusion des Dataframes

In [ ]:
df_films = df_films_clean.set_index("tconst")
df_titres = df_titre_clean.set_index("tconst")
df_films_titre = df_films.join(df_titres).reset_index()
df_films_titre.drop(columns=["tconst"], inplace=True)

del df_films_clean
del df_titre_clean

df_films_titre = df_films_titre.set_index("nconst")
df_noms_cleans = df_noms_clean.set_index("nconst")
df_final = df_films_titre.join(df_noms_cleans).reset_index()
df_final.drop(columns=["nconst"], inplace=True)

del df_noms_cleans
del df_titres

df_final = df_final.dropna()

df_final.to_csv("./data/df_final.csv")

del df_final

# La liste des films Américains (en gardant leur nom en français) et leur note moyenne

## Création des Dataframes

In [ ]:
df_note = pd.read_csv("./data/title.ratings.tsv", sep='\t')
df_pays = pd.read_table("./data/title.akas.tsv", sep='\t')
df_film = pd.read_table("./data/title.basics.tsv", sep='\t')

## Optimisation des Dataframes

In [ ]:
### Optimisation Dataframe "note"

df_note.info(memory_usage="deep")
df_note = df_note.drop(["numVotes"],axis=1)
df_note = df_note.dropna()

### Optimisation Dataframe "pays"

df_pays = df_pays.drop(["ordering","title", "language","types","attributes","isOriginalTitle"],axis=1)
df_pays = df_pays.dropna()
df_pays = df_pays[["titleId","region"]][df_pays["region"].str.contains("US")]
df_pays = df_pays.set_index("titleId")

### Optimisation Dataframe "film"

df_film = df_film.drop(["titleType", "primaryTitle","isAdult","startYear","endYear","runtimeMinutes","genres"], axis=1)
df_film = df_film.dropna()

## Fusion des Dataframes

In [ ]:
df_note = df_note.set_index("tconst")
df_film = df_film.set_index("tconst")
df_film_note = df_note.join(df_film).reset_index()
df_film_note.drop(columns=["tconst"], inplace=True)

df_film_note.to_csv("./data/df2_final.csv")

# Les notes moyennes des différents genres